<a href="https://colab.research.google.com/github/JohnPaul1278/ps3/blob/main/ps3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time, os, sys, re
import zipfile, json, datetime, string
import numpy as np
from statistics import *

import matplotlib.pyplot as plt

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler


from google.colab import files

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

plt.style.use('classic')

In [13]:
freedom = pd.ExcelFile('freedom.xlsx') #there are two sheets in this file but I only need one of them. I found this solution to define each separately.
df1 = pd.read_excel(xls, 'Index') #I don't think I'll need this in the merge itself
df2 = pd.read_excel(xls, 'FIW13-23') #THIS is the important one. This is the freedom index from Freedom House.

#here is the link to where I found this data: https://freedomhouse.org/report/freedom-world

#I don't anticipate using all of the data in this file. It's huge.
#I think I'll probably use data from specific years and countries based on what my other datasets have.
#I also don't know if I'll be looking at every type of freedom that is documented here. Some will be more interesting to compare than others.

df2

,Freedom in the World 2013-2023 Raw Data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43
0,Country/Territory,Region,C/T,Edition,Status,PR rating,CL rating,A1,A2,A3,...,F3,F4,F,G1,G2,G3,G4,G,CL,Total
1,Abkhazia,Eurasia,t,2023,PF,5,5,2,2,1,...,1,1,4,1,1,2,1,5,22,39
2,Afghanistan,Asia,c,2023,NF,7,7,0,0,0,...,0,0,0,0,1,0,1,2,7,8
3,Albania,Europe,c,2023,PF,3,3,3,3,3,...,2,3,9,3,2,2,2,9,39,67
4,Algeria,Africa,c,2023,NF,6,5,1,1,1,...,2,2,6,2,2,2,1,7,22,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,West Bank,Middle East,t,2013,NF,6,5,0,0,2,...,0,1,5,1,1,2,1,5,24,30
2302,Western Sahara,Africa,t,2013,NF,7,7,0,0,0,...,0,0,0,1,1,2,0,4,7,5
2303,Yemen,Middle East,c,2013,NF,6,6,1,0,2,...,0,1,2,2,1,1,1,5,16,25
2304,Zambia,Africa,c,2013,PF,3,4,3,3,3,...,2,2,8,3,2,1,2,8,34,62


In [46]:
df2 = df2.rename(columns={'Unnamed: 3': 'year', 'Unnamed: 4': 'Status', 'Unnamed: 5': 'PoliticalRights', 'Unnamed: 6': 'CivilLiberties', 'Freedom in the World 2013-2023 Raw Data': 'Countries' })
df3 = df2[df2.year == 2021]
df3 = df3[['Countries', 'year', 'Status', 'PoliticalRights', 'CivilLiberties']]
df3

,Countries,year,Status,PoliticalRights,CivilLiberties
421,Abkhazia,2021,PF,5,5
422,Afghanistan,2021,NF,5,6
423,Albania,2021,PF,3,3
424,Algeria,2021,NF,6,5
425,Andorra,2021,F,1,1
...,...,...,...,...,...
626,West Bank,2021,NF,7,5
627,Western Sahara,2021,NF,7,7
628,Yemen,2021,NF,7,6
629,Zambia,2021,PF,4,4


In [31]:
econfree=pd.read_csv('econfree.csv') #here is my second dataset. This is an economic freedom index from a different institution
#the slight change in perspective is interesting. I think many of the same questions are asked but the small differences between the two datasets may reveal something interesting
#here is where I found the dataset: https://www.fraserinstitute.org/economic-freedom/dataset?geozone=world&page=dataset&min-year=2&max-year=0&filter=0
econfree

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Area 1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1A,NaN,1B,...,5C,NaN,NaN,NaN,NaN,5D,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5Ci,5Cii,5Ciii,5Civ,NaN,5Di,5Dii,5Diii,NaN,NaN
3,NaN,Year,ISO Code,Countries,Economic Freedom Summary Index,Rank,Quartile,Government consumption,data,Transfers and subsidies,...,Regulatory Burden,Bureacracy costs,Impartial Public Administration,Tax compliance,Business regulations,Market openness,Business Permits,Distorton of the business environment,Freedom to enter markets and compete,Regulation
4,NaN,2021,AGO,Angola,5.38,149,4,7.18,15.59,9.70,...,2.94,4.67,4.90,4.64,4.29,3.10,7.92,1.25,4.09,4.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,NaN,2021,VNM,Vietnam,6.26,106,3,7.42,14.78,NaN,...,3.98,5.56,3.88,4.79,4.55,4.55,8.21,3.96,5.57,6.10
165,NaN,2021,YEM,"Yemen, Rep.",4.18,161,4,8.75,10.26,9.02,...,2.94,0.00,0.62,6.11,2.42,2.07,NaN,5.00,3.54,3.08
166,NaN,2021,ZAF,South Africa,6.53,94,3,4.66,24.16,7.87,...,3.36,4.22,6.34,7.57,5.37,5.30,8.39,3.75,5.81,6.35
167,NaN,2021,ZMB,Zambia,5.84,129,4,1.68,34.28,9.32,...,4.22,3.33,4.39,4.30,4.06,2.44,7.85,0.19,3.49,4.89


In [47]:
econfree = econfree.rename(columns={'Unnamed: 1': 'year', 'Unnamed: 3': 'countries', 'Unnamed: 4': 'efsi'})
econfree = econfree.rename(columns= {'countries': 'Countries'}) #I tried to change it without adding another row but it didn't work for some reason
econfree = econfree[['year', 'Countries', 'efsi']]
econfree = econfree[4::]
econfree

,year,Countries,efsi
16,2021,Bangladesh,5.81
17,2021,Bulgaria,7.46
18,2021,Bahrain,7.44
19,2021,"Bahamas, The",6.55
20,2021,Bosnia and Herzegovina,6.66
...,...,...,...
164,2021,Vietnam,6.26
165,2021,"Yemen, Rep.",4.18
166,2021,South Africa,6.53
167,2021,Zambia,5.84


In [55]:
merge1 = pd.merge(df3, econfree, on='Countries')
del merge1['year_x']
del merge1['year_y']
merge1

,Countries,Status,PoliticalRights,CivilLiberties,efsi
0,Algeria,NF,6,5,4.82
1,Bahrain,NF,7,6,7.44
2,Bangladesh,PF,5,5,5.81
3,Barbados,F,1,1,6.94
4,Belarus,NF,7,6,6.23
...,...,...,...,...,...
129,United States,F,2,2,8.14
130,Uruguay,F,1,1,7.06
131,Vietnam,NF,7,6,6.26
132,Zambia,PF,4,4,5.84


In [63]:
depression= pd.read_csv('depression.csv')
depression #this data only goes to 2019, whereas my earlier two datasets looked at data from 2021

,Entity,Code,Year,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,4.996118
1,Afghanistan,AFG,1991,4.989290
2,Afghanistan,AFG,1992,4.981346
3,Afghanistan,AFG,1993,4.976958
4,Afghanistan,AFG,1994,4.977782
...,...,...,...,...
6415,Zimbabwe,ZWE,2015,3.407624
6416,Zimbabwe,ZWE,2016,3.410755
6417,Zimbabwe,ZWE,2017,3.411965
6418,Zimbabwe,ZWE,2018,3.406929


array(['AFG', 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS', 'AUT',
       'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BHS',
       'BIH', 'BLR', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN', 'BWA',
       'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COG',
       'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU', 'DJI',
       'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH',
       'FIN', 'FJI', 'FRA', 'FSM', 'GAB', 'GBR', 'GEO', 'GHA', 'GIN',
       'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GTM', 'GUY', 'HND', 'HRV',
       'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR',
       'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KIR',
       'KOR', 'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO',
       'LTU', 'LUX', 'LVA', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MKD',
       'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI',
       'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL',
       'OMN', 'PAK',